In [1]:
import os
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm, trange

from lib.data import conditions as cond
from lib.data import tables as tab
from lib import bootstrap as bs
from lib.features import correlation as corr
from lib.features import decomposition as de

In [2]:
engine = tab.sa.create_engine(os.environ['SQLALCHEMY_ENGINE_URL'])
Session = tab.sa.orm.sessionmaker(bind=engine)
session = Session()

In [3]:
def get_samplers(session, celltype) -> ("mouseid_sampler", "averagetr_sampler"):
    sample_mouseids = bs.get_mouseid_sampler(session, celltype)
    sample_cellids = {
        mid: (lambda mid=mid: bs.get_day_one_cell_ids(session, mid))
        for mid in bs.get_mouseids_by_celltype(session, celltype)
    }
    sample_averagetr = {
        day: {
            mid: bs.get_averagetrace_given_cellid_sampler(session, sample_cellids[mid], mid, day)
            for mid in bs.get_mouseids_by_celltype(session, celltype)
        }
        for day in (1, 7)
    }
    
    return sample_mouseids, sample_averagetr

In [4]:
trial_structure = de.TrialBasisFunctions(
    390, tone_duration=1.0, delay_duration=1.5, reward_duration=2.5
)
trial_bases = {
    'tone + delay': trial_structure.tone + trial_structure.delay,
    'reward': trial_structure.reward
}

In [ ]:
def df_to_csv_confirm_overwrite(df: pd.DataFrame, fname: str):
    if not os.path.exists(fname):
        df.to_csv(fname)
    else:
        if input('Overwrite? (yn)').lower() == 'y':
            df.to_csv(fname)

Run bootstrap. Average shuffle traces are not Z-scored, but this doesn't matter because spearman correlation is scale invariant.

In [5]:
NUM_BOOTSTRAPS = 1000

# Pyramidal neurons

In [6]:
sample_mouseids, sample_averagetr = get_samplers(session, cond.CellType.pyr)

In [7]:
bootstrap_records = []
for i in trange(NUM_BOOTSTRAPS, desc='Bootstrap samples', leave=True):
    mouseids = sample_mouseids()
    
    for label, basis in trial_bases.items():
        
        sp_stats_d1 = []
        sp_stats_d7 = []
        sp_stats_delta = []
        for mid in tqdm(mouseids, desc='Mice', leave=False):
            d1_tmp = corr.vectorized_spearman_corr(sample_averagetr[1][mid](), basis)
            d7_tmp = corr.vectorized_spearman_corr(sample_averagetr[7][mid](), basis)
            
            sp_stats_d1.append(np.nanmean(d1_tmp))
            sp_stats_d7.append(np.nanmean(d7_tmp))
            sp_stats_delta.append(np.nanmean(d7_tmp - d1_tmp))  # Cell by cell change in tuning. Assumes sample_averagetr returns cells in same order.
            
        bootstrap_records.append(
            {'bs_sample': i, 'day': '1', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d1)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': '7', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d7)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': 'delta', 'trial_component': label, 'spearman': np.nanmean(sp_stats_delta)}
        )

In [8]:
bs_df = pd.DataFrame(bootstrap_records)
bs_df

,bs_sample,day,trial_component,spearman
0,0,1,tone + delay,-0.003288
1,0,7,tone + delay,0.012744
2,0,delta,tone + delay,0.016032
3,0,1,reward,0.010230
4,0,7,reward,0.002099
...,...,...,...,...
5995,999,7,tone + delay,0.026180
5996,999,delta,tone + delay,0.013238
5997,999,1,reward,-0.036114
5998,999,7,reward,0.023421


In [9]:
df_to_csv_confirm_overwrite(bs_df, '../../interneuron-reward-data/processed/pyr_corr_bootstrap.csv')

# VIP neurons

In [10]:
sample_mouseids, sample_averagetr = get_samplers(session, cond.CellType.vip)

NaNs are introduced because one cell is missing from day 7 in VIP mouse CL146. It shows up as a row of NaNs in `sample_averagetr[7]['CL146']()`. Using `nanmean` here prevents this from crashing the bootstrap and doesn't affect the null distribution (except that the null distributions for day 7 and change are based on one less mouse, of course). Don't actually know which cell this is because cell ids are returned from `bs.get_day_one_cell_ids()` in no particular order.

In [11]:
bootstrap_records = []
for i in trange(NUM_BOOTSTRAPS, desc='Bootstrap samples', leave=True):
    mouseids = sample_mouseids()
    
    for label, basis in trial_bases.items():
        
        sp_stats_d1 = []
        sp_stats_d7 = []
        sp_stats_delta = []
        for mid in tqdm(mouseids, desc='Mice', leave=False):
            d1_tmp = corr.vectorized_spearman_corr(sample_averagetr[1][mid](), basis)
            d7_tmp = corr.vectorized_spearman_corr(sample_averagetr[7][mid](), basis)
            
            sp_stats_d1.append(np.nanmean(d1_tmp))
            sp_stats_d7.append(np.nanmean(d7_tmp))
            sp_stats_delta.append(np.nanmean(d7_tmp - d1_tmp))  # Cell by cell change in tuning. Assumes sample_averagetr returns cells in same order.
            
        bootstrap_records.append(
            {'bs_sample': i, 'day': '1', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d1)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': '7', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d7)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': 'delta', 'trial_component': label, 'spearman': np.nanmean(sp_stats_delta)}
        )

<ipython-input-11-5eb2fae2e9c3>:15: RuntimeWarning: Mean of empty slice
  sp_stats_d7.append(np.nanmean(d7_tmp))
<ipython-input-11-5eb2fae2e9c3>:16: RuntimeWarning: Mean of empty slice
  sp_stats_delta.append(np.nanmean(d7_tmp - d1_tmp))  # Cell by cell change in tuning. Assumes sample_averagetr returns cells in same order.


In [12]:
bs_df = pd.DataFrame(bootstrap_records)
bs_df

,bs_sample,day,trial_component,spearman
0,0,1,tone + delay,0.018945
1,0,7,tone + delay,-0.018822
2,0,delta,tone + delay,-0.037767
3,0,1,reward,-0.016652
4,0,7,reward,-0.029357
...,...,...,...,...
5995,999,7,tone + delay,-0.002298
5996,999,delta,tone + delay,-0.012099
5997,999,1,reward,-0.004117
5998,999,7,reward,-0.008011


In [13]:
df_to_csv_confirm_overwrite(bs_df, '../../interneuron-reward-data/processed/vip_corr_bootstrap.csv')

Overwrite? (yn)y


# PV

In [14]:
sample_mouseids, sample_averagetr = get_samplers(session, cond.CellType.pv)

In [15]:
bootstrap_records = []
for i in trange(NUM_BOOTSTRAPS, desc='Bootstrap samples', leave=True):
    mouseids = sample_mouseids()
    
    for label, basis in trial_bases.items():
        
        sp_stats_d1 = []
        sp_stats_d7 = []
        sp_stats_delta = []
        for mid in tqdm(mouseids, desc='Mice', leave=False):
            d1_tmp = corr.vectorized_spearman_corr(sample_averagetr[1][mid](), basis)
            d7_tmp = corr.vectorized_spearman_corr(sample_averagetr[7][mid](), basis)
            
            sp_stats_d1.append(np.nanmean(d1_tmp))
            sp_stats_d7.append(np.nanmean(d7_tmp))
            sp_stats_delta.append(np.nanmean(d7_tmp - d1_tmp))  # Cell by cell change in tuning. Assumes sample_averagetr returns cells in same order.
            
        bootstrap_records.append(
            {'bs_sample': i, 'day': '1', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d1)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': '7', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d7)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': 'delta', 'trial_component': label, 'spearman': np.nanmean(sp_stats_delta)}
        )

In [16]:
bs_df = pd.DataFrame(bootstrap_records)
bs_df

,bs_sample,day,trial_component,spearman
0,0,1,tone + delay,0.042191
1,0,7,tone + delay,-0.015328
2,0,delta,tone + delay,-0.057519
3,0,1,reward,-0.031407
4,0,7,reward,-0.004633
...,...,...,...,...
5995,999,7,tone + delay,-0.002442
5996,999,delta,tone + delay,0.012694
5997,999,1,reward,-0.014329
5998,999,7,reward,-0.008304


In [17]:
df_to_csv_confirm_overwrite(bs_df, '../../interneuron-reward-data/processed/pv_corr_bootstrap.csv')

Overwrite? (yn)y


# SOM

In [18]:
sample_mouseids, sample_averagetr = get_samplers(session, cond.CellType.som)

In [19]:
bootstrap_records = []
for i in trange(NUM_BOOTSTRAPS, desc='Bootstrap samples', leave=True):
    mouseids = sample_mouseids()
    
    for label, basis in trial_bases.items():
        
        sp_stats_d1 = []
        sp_stats_d7 = []
        sp_stats_delta = []
        for mid in tqdm(mouseids, desc='Mice', leave=False):
            d1_tmp = corr.vectorized_spearman_corr(sample_averagetr[1][mid](), basis)
            d7_tmp = corr.vectorized_spearman_corr(sample_averagetr[7][mid](), basis)
            
            sp_stats_d1.append(np.nanmean(d1_tmp))
            sp_stats_d7.append(np.nanmean(d7_tmp))
            sp_stats_delta.append(np.nanmean(d7_tmp - d1_tmp))  # Cell by cell change in tuning. Assumes sample_averagetr returns cells in same order.
            
        bootstrap_records.append(
            {'bs_sample': i, 'day': '1', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d1)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': '7', 'trial_component': label, 'spearman': np.nanmean(sp_stats_d7)}
        )
        bootstrap_records.append(
            {'bs_sample': i, 'day': 'delta', 'trial_component': label, 'spearman': np.nanmean(sp_stats_delta)}
        )

In [20]:
bs_df = pd.DataFrame(bootstrap_records)
bs_df

,bs_sample,day,trial_component,spearman
0,0,1,tone + delay,0.002726
1,0,7,tone + delay,0.010727
2,0,delta,tone + delay,0.008002
3,0,1,reward,0.064035
4,0,7,reward,0.003483
...,...,...,...,...
5995,999,7,tone + delay,-0.005431
5996,999,delta,tone + delay,-0.066335
5997,999,1,reward,-0.003432
5998,999,7,reward,0.021252


In [21]:
df_to_csv_confirm_overwrite(bs_df, '../../interneuron-reward-data/processed/som_corr_bootstrap.csv')